In [1]:
import cv2
import os
import numpy as np

input_file = 'VDB\.mp4'   

# Open Video

In [2]:
cap = cv2.VideoCapture(input_file)
if not cap.isOpened():
    print("Error: Could not open video.")

# Get Video Properties

In [3]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Width: ",width)
print("Height: ",height)
print("Original FPS: ",original_fps)
print("Frame Count: ",frame_count)

Width:  640
Height:  360
Original FPS:  24.0
Frame Count:  16627


In [4]:
def get_frames(cap):
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(frame_count):
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            return frames

        # Save the frame to the list
        frames.append(frame)
        
    if not frames:
        print("No frames were saved.")
    
    return frames

In [5]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
Frames = get_frames(cap)

# Delete PNG Files in Directory

In [6]:
def delete_png_files(directory_path):
    # List all files in the specified directory
    for filename in os.listdir(directory_path):
        # Construct full file path
        file_path = os.path.join(directory_path, filename)
        try:
            # Check if it's a PNG file and remove it
            if os.path.isfile(file_path) and filename.lower().endswith('.png'):
                os.remove(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Read and Save Frames in Interval

In [7]:
def save_frames_as_vid(frames, frame_folder):
    # Create the folder to save frames if it doesn't exist
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)

    frame_count = 0
    for i in range(len(frames)):
        # Save the frame as an image file
        frame_filename = os.path.join(frame_folder, f'frame_{frame_count:03d}.png')
        cv2.imwrite(frame_filename, frames[i])
        frame_count += 1

    #if not frames:
         #print("No frames were saved.")

In [8]:
delete_png_files('saved_frames')

In [9]:
save_frames_as_vid(Frames[50:100],"saved_frames")

# Create Video with Set of Frames

In [10]:
def save_vid(frames,output_file,fps):
    # Get frame dimensions
    height, width, _ = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Write the frames to the new video file
    [out.write(frame) for frame in frames];

    # Release the video writer object
    out.release()

In [11]:
save_vid(Frames[100:150], "slice_vid.mp4", original_fps)

# Change FPS

In [12]:
save_vid(Frames[100:150],"slice_vid_new_fps.mp4",12)

# Play Frames

In [13]:
def play_frames(frames,fps):
    
    delay = int(1000/fps)
    print("Delay: ",delay)

    for frame in frames:
        # Display the frame
        cv2.imshow('Video Playback', frame)
        # Exit the playback if 'q' is pressed
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cv2.destroyAllWindows()

In [14]:
play_frames(Frames[:100],24)

Delay:  41


# Optical Flow

## Farneback Magnitude

In [15]:
def OFM(frames):
    
    OF = [] 
    #Convert to Grayscale
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)

    for i in range(len(frames)-1):
        #Convert the next frame to grayscale
        next_gray = cv2.cvtColor(frames[i+1], cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5,  3,  10,  10,    5,    1.2,   0)
                                            #prev      next       flow  dist lvl  win  it  smooth  std   flag 
 
        # Visualize the optical flow
        hsv = np.zeros_like(frames[i]) #[ch0,ch1,ch2]
        hsv[..., 1] = 255 #ch1 Saturation (Full)

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1]) #Cartesian to Polar
        hsv[..., 0] = ang * 180 / np.pi / 2  
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) #Normalize from 0 to 255
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)   #Convert to RBG
        OF.append(flow_rgb)
        
        flow_M  = flow[...,0]+flow[...,1]
        
        # Display the original frame and the optical flow
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        cv2.imshow('Optical Flow', flow_rgb)
        cv2.imshow('Optical Flow Mag',flow_M)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF==ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray

    # Release the video capture object and close all OpenCV windows
    cv2.destroyAllWindows()
    return OF

In [16]:
Optical_R = OFM(Frames[40:60])

## Farneack with Vectors

In [377]:
def draw_optical_flow_vectors(flow, frame, step):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # Create a mask to draw the vectors
    mask = np.zeros_like(frame)
    
    # Create line endpoints
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    # Draw lines and circles
    for (x1, y1), (x2, y2) in lines:
        cv2.line(mask, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), -1)

    return cv2.add(frame, mask)

def OFV(frames,step):
    OF = []
    
    #convert it to grayscale
    prev_gray = cv2.cvtColor(np.copy(frames[0]), cv2.COLOR_BGR2GRAY)

    for i in range(1,len(frames)):
        # next frame convert it to grayscale
        next_gray = cv2.cvtColor(np.copy(frames[i]), cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Draw the optical flow vectors on the frame
        flow_frame = draw_optical_flow_vectors(flow, np.copy(frames[i]),step)
        OF.append(flow)
        
        # Display the original frame with optical flow vectors
        cv2.imshow('Optical Flow Vectors', flow_frame)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray

    # Release the video capture object and close all OpenCV windows
    cv2.destroyAllWindows()
    return OF

In [378]:
Optical_V = OFV(Frames[50:100],20)

In [379]:
play_frames(Frames[50:100],24)

Delay:  41


## Lucas-Kanade

In [327]:
def OFKL(frames):
    # Parameters for the Lucas-Kanade optical flow
    lk_params = dict(winSize  = (15, 15),
                     maxLevel = 2,
                     criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners = 100,
                          qualityLevel = 0.3,
                          minDistance = 7,
                          blockSize = 7)

    # Take first frame and find corners in it
    old_frame = np.copy(frames[0])
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    # Check if any points are detected
    if p0 is None:
        print("No points detected in the initial frame.")
        return

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    for i in range(1, len(frames)):
        frame = np.copy(frames[i])
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow
        if p0 is not None and len(p0) > 0:
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Draw the tracks
            for j, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
                frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

            img = cv2.add(frame, mask)
            
            cv2.imshow('Prev Frame',frames[i-1])
            cv2.imshow('Next Frame',frames[i])
            cv2.imshow('Optical Flow', img)
            
            if cv2.waitKey(0) & 0xff==ord('q'):
                break

            # Update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)
        else:
            print("No good points to track.")
            break

    cv2.destroyAllWindows()

In [328]:
OFKL(Frames[50:100])

No good points to track.


# Frames Differences

In [335]:
def frame_dif(frames, threshold=30):
    D = []
    prev_frame = frames[0]

    for i in range(1,len(frames)):

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        # Compute absolute difference between frames
        diff = cv2.absdiff(prev_gray, curr_gray)
              
        # Create a mask to highlight pixels with significant changes
        mask = np.zeros_like(diff)
        mask[diff > threshold] = 255
        D.append(mask)
        # Show original frame and mask
        cv2.imshow('Original Frame', frames[i])
        cv2.imshow('Pixels with Most Changes', mask)
        if cv2.waitKey(0) & 0xff == ord('q'):
            break

        prev_frame = frames[i]

    # Release the video capture object
    cv2.destroyAllWindows()
    return D

In [336]:
Dif = frame_dif(Frames[50:100],30)